# Aditya Sawant's Version of SPN_IP_5Shot.ipynb

### Libraries Used

- tensorflow
- sklearn
- numpy
- matplotlib
- pandas
- scipy
- tensorflow_probability


## Initial setup

### Install Libraries

In [1]:
# %pip install scikit-learn numpy matplotlib scipy tensorflow_probability pandas tqdm plotly
# tensorflow[and-cuda]==2.10 [cuDNN 8.1.1 CUDA 11.2]

### Import Libraries

In [2]:
import random
import statistics
import os
import importlib
import IPython
from IPython.display import clear_output
  
from operator import truediv

import tensorflow as tf
print(tf.version.GIT_VERSION, tf.version.VERSION)
print(tf.test.is_built_with_cuda())
print("Number of GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Number of Devices Available: ", len(tf.config.experimental.list_physical_devices()))
# import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from matplotlib import patches as matPatch
from tensorflow.python.keras import backend as K

from tensorflow.keras import Sequential, layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.compat.v1.distributions import Bernoulli

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
import spectral
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
from datetime import date, datetime
from tqdm.auto import tqdm

from encoders.conv_gan import createModel
from lib.PrototypicalNetwork import Prototypical
from lib.misc import timeIt, plotData
from lib.Data import Data
from lib.Stats import Stats
from lib.Predict import *


v2.10.0-rc3-6-g359c3cdfc5f 2.10.0
True
Number of GPUs Available:  1
Number of Devices Available:  2


d:\HSI FSL BE-10 Major Project\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



## Global Variables

In [3]:




# Test each code block individually




TEST_BLOCKS: bool = False
CWD: str = os.getcwd()


VERBOSE: bool = False
SAVE_REPORT: bool = True


SAVE_MODEL: bool = False


# Data Loading and Preprocessing


# Dataset Used : Indian Pines


DATASET: str = 'IP' # IP (indian_pines) PU (pavia_university) SA (salinas) HU (houston) 


PATH_TO_DATASET: str = CWD + '\\Datasets\\'# PCA


PCA_COMPONENTS: int = 30 # Number of components to keep after PCA reduction# Window size for forming image cubes


WINDOW_SIZE: int = 11# Image dimensions after forming image cubes


IMAGE_WIDTH: int


IMAGE_HEIGHT: int


IMAGE_DEPTH: int


IMAGE_CHANNEL: int 


IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_DEPTH, IMAGE_CHANNEL = 11, 11, 30, 1
IMAGE_DATA = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_DEPTH, IMAGE_CHANNEL)



# Model Parameters



N_TIMES = 25 # Number of times to run the model. Internally, the model is runs each episode N_TIMES times# Learning Rate


LEARNING_RATE: float = 0.00001

# Temprature Scaling


TAU: float = 1.8


# C (No. of Classes) K (No. of Samples per Class) N (No. of Patches per Class)


TRAIN_C: int = 5 # Number of classes to be used for training


TRAIN_K: int = 5 # Number of patches per class to be used for support during training


TRAIN_N: int = 15 # Number of patches per class to be used for query during training

TUNE_C: int = 3 # Number of classes to be used for tunning


TUNE_K: int = 1 # Number of patches per class to be used for support during tunning


TUNE_N: int = 4 # Number of patches per class to be used for query during tunning

TEST_C: int = 3 # Number of classes to be used for testing


TEST_K: int = 5 # Number of patches per class to be used for support during testing


TEST_N: int = 15 # Number of patches per class to be used for query during testing#
# ===================================


# DO NOT REMOVE THIS.

# ===================================


# Don't know this yet, probably used in the model to calculate loss


MC_LOSS_WEIGHT: int = 5 


# DIRECTLY USED IN PROTOTYPICAL NETWORK CLASS IN TESTING CASE


# ===================================
# Training Epochs 50


TRAINING_EPOCH: int = 40
# Training Episode 100


TRAINING_EPISODE: int = 100
# Tunning Epochs 41


TUNNING_EPOCH: int = 15
# Tunning Episode 100


TUNNING_EPISODE: int = 100
# Testing Epochs 1000


TESTING_EPOCH: int = 1000
# Metrics to be used for evaluation


PRE_TUNE_TESTING: bool = True


train_loss = tf.metrics.Mean(name='train_loss')


train_acc = tf.metrics.Mean(name='train_accuracy')


tune_loss = tf.metrics.Mean(name='tune_loss')


tune_acc = tf.metrics.Mean(name='tune_accuracy')


test_loss = tf.metrics.Mean(name='test_loss')


test_acc = tf.metrics.Mean(name='test_accuracy')

trainingData = []


tunningData = []


testingData = []
TABLE= ''

run_folder =  f'{date.today()}' + '-' + f'{datetime.now().hour}_5_1' + '\\' 
checkpoint_dir = CWD + '\\saves\\' + run_folder + DATASET + '\\' + f'{TRAIN_K}_shot_way' + '\\Train'


checkpoint_prefix_train = os.path.join(checkpoint_dir, "ckpt")
checkpoint_dir1 = CWD + '\\saves\\' + run_folder + DATASET + '\\' + f'{TRAIN_K}_shot_way' + '\\Train\\Tune'


checkpoint_prefix_tune = os.path.join(checkpoint_dir1, "ckpt")
report_path = CWD + f'\\Reports\\Report_{date.today()}_{str(datetime.now()).split(".")[0].split()[1].replace(":", "-")}.txt'
report_path_pre = CWD + f'\\Reports\\Report_{date.today()}_{str(datetime.now()).split(".")[0].split()[1].replace(":", "-")}.txt'


model_save_path = CWD + '\\saves\\' + run_folder + DATASET + '\\' + f'{TRAIN_K}_shot_way' + '\\Train\\encoder.h5'



finalReportPath: str = ''
reportTotal_path: str = ''
reportTotal_path_pre: str = ''
predict_path: str = ''
predictTrue_path: str = ''
gt_path: str = ''
current_path: str = ''

checkpoint = None  # To be used for loading checkpoints. Declared in the Main Block


ProtoModel = None  # Prototypical Network Object. Declared in the Main Block


model = None  # Model Object. Declared in the Main Block


optimizer = None  # Optimizer Object. Declared in the Main Block

X = None
Y = None
patches = None
target_names = None

## Data Loaders for Model

### For Training

In [4]:
def createTrainingEpisode(patches:list, labels:list, K:int, C:int, N:int ):
    """
    createTrainingEpisode creates a training episode for the N-way K-shot learning task.
    
    :param patches: list of all patches classified into different classes.
    :param labels: list of classes from which the traning episode is to be created.
    :param K: number of patches per class in the support set.
    :param C: number of classes in the training episode.
    :param N: number of patches per class in the query set.
    :return queryPatches, queryLabels, supportPatches, supportLabels: training episode
    
    Algorithm:
    - Select N classes from the list of labels. They should be unique.
    - For each class, select K+Q patches. They should be unique.
        - First K patches are support patches.
        - Last Q patches are query patches.
        - Append the support patches to supportPatches.
        - Append the query patches to queryPatches.
        - Append the class label to queryLabels Q times.
    - Shuffle the queryPatches and queryLabels in the same order.
    - Convert the queryPatches and supportPatches to tensors.
    
    """
    
    selectedLabels = random.sample(labels, C)
    supportPatches = []
    supportLabels = list(selectedLabels)
    queryPatches = []
    queryLabels = []
    
    for n in selectedLabels:
        sran_indices = np.random.choice(len(patches[n-1]),K,replace=False)  # for class no X-1: select K samples 
        supportPatches.extend( patches[n-1][sran_indices,:,:,:,:])
        qran_indices = np.random.choice(len(patches[n-1]),N,replace=False)  # N Samples for Query
        queryPatches.extend(patches[n-1][qran_indices,:,:,:,:])
        queryLabels.extend([n]*N)
    
    shuffled = list(zip(queryPatches, queryLabels))
    random.shuffle(shuffled)
    queryPatches, queryLabels = zip(*shuffled)
    
    queryPatches = tf.convert_to_tensor(np.reshape(np.asarray(queryPatches),(C*N,IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_DEPTH,IMAGE_CHANNEL)),dtype=tf.float32)
    supportPatches = tf.convert_to_tensor(np.reshape(np.asarray(supportPatches),(C*K,IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_DEPTH,IMAGE_CHANNEL)),dtype=tf.float32)
    
    return queryPatches, queryLabels, supportPatches, supportLabels


In [5]:
def train_step(support, query, support_labels, query_labels, K, C, N):
    # Forward & update gradients
    with tf.GradientTape() as tape:
        loss, mean_accuracy, mean_predictions = ProtoModel(support, query, support_labels, query_labels, K, C, N,training=True)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    # A gradient simply measures the change in all weights with regard to the change in error. You can also think of a gradient as the slope of a function. The higher the gradient, the steeper the slope and the faster a model can learn. But if the slope is zero, the model stops learning
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Log loss and accuracy for step
    train_loss(loss)
    train_acc(mean_accuracy)


In [6]:
@timeIt
def trainingEpochs(patches, labels, n_epochs, n_episodes):
    """
    trainingEpochs function trains the model for n_epochs and n_episodes.
    
    :param patches: image patches to be trained
    :param labels: corresponding labels to be used
    :param n_epochs: number of epochs
    :param n_episodes: number of episodes
    :return: None
    """
    
    template = 'Epoch {}/{}, Episode {}/{}, Train Loss: {:.2f}, Train Accuracy: {:.2f}'
    # for epoch in tqdm(range(n_epochs), desc='Epochs'):
    #     train_loss.reset_states()
    #     train_acc.reset_states()
    #     for episode in tqdm(range(n_episodes), desc=f'Episodes (Loss: {l:.2f}, Acc: {a:.2f})'):
    
    break_after_epoch = False
    no_of_epoch_before_break = 1
    trainObj = tqdm(total=n_episodes * n_epochs, desc=f'Epoch 1/{n_epochs}, Episode 1/{n_episodes}')
    for epoch in range(n_epochs):
        train_loss.reset_states()
        train_acc.reset_states()
        for episode in range(n_episodes):
            queryPatches, queryLabels, supportPatches, supportLabels = createTrainingEpisode(patches, labels, TRAIN_K, TRAIN_C, TRAIN_N)
            train_step(supportPatches, queryPatches,supportLabels,  queryLabels, TRAIN_K, TRAIN_C, TRAIN_N)
            # clear_output(wait=True)
            if(train_acc.result().numpy()*100 >= 100):
                break_after_epoch = True
            trainObj.set_description(
                f'Epoch {epoch+1}/{n_epochs}, Episode {episode+1}/{n_episodes} (Loss: {train_loss.result().numpy():.2f}, Acc: {train_acc.result().numpy()*100:.2f})')
            trainObj.update(1)
            if(VERBOSE):
                print(template.format(epoch+1, n_epochs, episode+1, n_episodes, train_loss.result()*100, train_acc.result()*100))
                trainingData.append([train_loss.result(),  train_acc.result()*100])
                plotData(trainingData)
        # TO BREAK IF ACC REACHES 100
        # if(break_after_epoch):
        #     if(no_of_epoch_before_break == 0):
        #         trainObj.close()
        #         print()
        #         print('Stoping Training as 100% Acc reached atleast once')
        #         checkpoint.save(file_prefix=checkpoint_prefix_train)    
        #         break
        #     no_of_epoch_before_break -= 1

        if(epoch and epoch % 5 == 0):
            checkpoint.save(file_prefix=checkpoint_prefix_train)    
    trainObj.close()
        

### For Tuning

In [7]:
def createTunningEpisodes(patches:list, labels:list, K:int, C:int, N:int):
    """
    createTuningEpisodes creates a tuning episode for the N-way K-shot learning task.
    
    :param patches: list of all patches classified into different classes.
    :param labels: list of classes from which the tuning episode is to be created.
    :param K: number of patches per class in the support set.
    :param C: number of classes in the tuning episode.
    :param N: number of patches per class in the query set.
    :return queryPatches, queryLabels, supportPatches, supportLabels: tuning episode
    
    Algorithm:
    - Select C classes from the list of labels. They should be unique.
    - For each selected class.
        - Shuffle the patches of that class.
        - First K patches are support patches.
        - Next N patches are query patches. 
        - Append the support patches to supportPatches.
        - Append the query patches to queryPatches.
        - Append the class label to queryLabels N times.
    - Shuffle the queryPatches and queryLabels in the same order.
    - Convert the queryPatches and supportPatches to tensors.
    
    """

    selected_classes = np.random.choice(labels,C,replace=False)
    supportLabels  = list(selected_classes)
    queryLabels = []
    supportPatches = []
    queryPatches = []
    
    for x in selected_classes :
        y = labels.index(x)
        np.random.shuffle(patches[y])    
        supportPatches.extend(patches[y][:K,:,:,:,:])  # 1st K patches for support set
        queryPatches.extend(patches[y][K:K+N,:,:,:,:])   # next N patches for query set
        queryLabels.extend([x]*N)            
          # next 5 labels for query set
    
    shuffled = list(zip(queryPatches, queryLabels))
    random.shuffle(shuffled)
    queryPatches, queryLabels = zip(*shuffled)
    
    queryPatches = tf.convert_to_tensor(np.reshape(np.asarray(queryPatches),(C*N,IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_DEPTH,IMAGE_CHANNEL)),dtype=tf.float32)
    supportPatches = tf.convert_to_tensor(np.reshape(np.asarray(supportPatches),(C*K,IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_DEPTH,IMAGE_CHANNEL)),dtype=tf.float32)
    
    return queryPatches, queryLabels, supportPatches, supportLabels
    

In [8]:
def tune_step(support, query, support_labels, query_labels, K, C, N):
    # Forward & update gradients
    with tf.GradientTape() as tape:
        loss, mean_accuracy, mean_predictions = ProtoModel(support, query, support_labels, query_labels, K, C, N,training=True)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Log loss and accuracy for step
    tune_loss(loss)
    tune_acc(mean_accuracy)

In [9]:
@timeIt
def tunningEpochs(patches, labels, n_epochs, n_episodes):
    """
    trainingEpochs function trains the model for n_epochs and n_episodes.
    
    :param patches: image patches to be trained
    :param labels: corresponding labels to be used
    :param n_epochs: number of epochs
    :param n_episodes: number of episodes
    :return: None
    """
    template = 'Epoch {}/{}, Tune Loss: {:.2f}, Tune Accuracy: {:.2f}'

    epochObj = tqdm(range(n_epochs), desc='Epochs')
    for epoch in epochObj: 
        tune_loss.reset_states()  
        tune_acc.reset_states()    
        for epi in range(n_episodes):  
            queryPatches, queryLabels, supportPatches, supportLabels = createTunningEpisodes(patches, labels, TUNE_K, TUNE_C, TUNE_N)    
            tune_step(supportPatches, queryPatches,supportLabels, queryLabels, TUNE_K, TUNE_C, TUNE_N)   
            # clear_output(wait=True)   
        epochObj.set_postfix(
            {'Loss': tune_loss.result().numpy(), 'Acc': tune_acc.result().numpy()*100}, refresh=True)
        if(VERBOSE):
            print(template.format(epoch+1, n_epochs,tune_loss.result(),tune_acc.result()*100))
            tunningData.append([tune_loss.result(),  tune_acc.result()*100])
            plotData(tunningData)
        if (epoch+1)%5 == 0 :
            checkpoint.save(file_prefix = checkpoint_prefix_tune) 
    epochObj.close()

### For Testing

In [10]:
def createTestingEpisode(patches, labels, K, C, N, i, f):
    selected_classes = labels[i:f]   # [1, 2, 3, 4, 5, 6, 7, 8]
    support_labels = list(selected_classes)
    query_labels = []
    support_patches = []
    query_patches = []
    for x in selected_classes:
        y = labels.index(x)
        # for class no X-1: select K samples
        sran_indices = np.random.choice(len(patches[y]), K, replace=False)
        support_patches.extend(patches[y][sran_indices, :, :, :, :])
        qran_indices = np.random.choice(
            len(patches[y]), N, replace=False)  # N Samples for Query
        query_patches.extend(patches[y][qran_indices, :, :, :, :])
        query_labels.extend([x]*N)

    
        
        # support_imgs = patches[y][:K, :, :, :, :]
        # query_imgs = patches[y][K: K + N, :, :, :, :]
        # support_patches.extend(support_imgs)
        # query_patches.extend(query_imgs)
        # for i in range(query_imgs.shape[0]):
        #     query_labels.append(x)
            
            
            
            
    temp1 = list(zip(query_patches, query_labels))
    random.shuffle(temp1)
    query_patches, query_labels = zip(*temp1)
    x = len(query_labels)
    query_patches = tf.convert_to_tensor(np.reshape(np.asarray(
        query_patches), (x, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, IMAGE_CHANNEL)), dtype=tf.float32)
    support_patches = tf.convert_to_tensor(np.reshape(np.asarray(support_patches), (
        C*K, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, IMAGE_CHANNEL)), dtype=tf.float32)
    return query_patches, support_patches, query_labels, support_labels, x

In [11]:
def test_step(support, query, support_labels, query_labels, K, C, y):
    loss, mc_predictions, mean_accuracy, classwise_mean_acc, y = ProtoModel(support, query, support_labels, query_labels, K, C, y,training=False)
    return loss, mc_predictions, mean_accuracy, classwise_mean_acc, y

In [12]:
@timeIt
def testingEpochs(patches, labels, n_epochs):
    """
    testingEpochs function tests the model for n_epochs.

    :param patches: image patches to be trained
    :param labels: corresponding labels to be used
    :param n_epochs: number of epochs
    :return: None
    """
    epochObj = tqdm(range(n_epochs), desc=f'Epochs')

    all_y_pred1 = []
    all_y_label1 = []
    all_y_pred2 = []
    all_y_label2 = []
    for epoch in epochObj:
        test_loss.reset_states()
        test_acc.reset_states()

        tquery_patches1, tsupport_patches1, query_labels1, support_labels1, x1 = createTestingEpisode(
            patches, labels, TEST_K, TEST_C, TEST_N, 0, 3)
        loss1, mc_predictions1, mean_accuracy1, classwise_mean_acc1, y1 = test_step(
            tsupport_patches1, tquery_patches1, support_labels1, query_labels1, TEST_K, TEST_C, TEST_N)
        oa1 = mean_accuracy1.numpy()

        all_y_pred1.append(tf.convert_to_tensor(mc_predictions1))
        all_y_label1.append(tf.convert_to_tensor(y1))

        if (DATASET == 'IP' or DATASET == 'SA'):
            tquery_patches2, tsupport_patches2, query_labels2, support_labels2, x2 = createTestingEpisode(
                patches, labels, TEST_K, TEST_C, TEST_N, 3, 6)
            loss2, mc_predictions2, mean_accuracy2, classwise_mean_acc2, y2 = test_step(
                tsupport_patches2, tquery_patches2, support_labels2, query_labels2, TEST_K, TEST_C, TEST_N)
            oa2 = mean_accuracy2.numpy()

            all_y_pred2.append(tf.convert_to_tensor(mc_predictions2))
            all_y_label2.append(tf.convert_to_tensor(y2))
        oa2 = oa2 if (DATASET == 'IP' or DATASET == 'SA') else 0
        epochObj.set_postfix({'OA1': f'{oa1}', 'OA2': f'{oa2}'}, refresh=True)
        
        if (VERBOSE):
            print("=========================================")
            print(f"Epoch {epoch+1}/{n_epochs}")
            print("-----------------------------------------")
            print(f"Overall Accuracy 1 (OA1): {mean_accuracy1}")
            # Class Wise Accuracy
            for i in range(TEST_C):
                print(f"Class {i+1} Accuracy: {classwise_mean_acc1[i]}")
            print(f"Loss: {loss1.numpy():.3f}")
            if (DATASET == 'IP' or DATASET == 'SA'):
                print("=========================================")
            else:
                print("-----------------------------------------")
                print(f"Overall Accuracy 2 (OA2): {mean_accuracy2}")
                # Class Wise Accuracy
                for i in range(TEST_C):
                    print(
                        f"Class {i+1+TEST_C} Accuracy: {classwise_mean_acc2[i]}")
                print(f"Loss: {loss2.numpy():.3f}")
                print("=========================================")

            testingData.append([mean_accuracy1*100, mean_accuracy2 *
                               100, loss1.numpy(), 0 if loss2 == 0 else loss2.numpy()])
            plotData(testingData, testing=True)

    epochObj.close()
    try:
        combined_y_pred1 = tf.concat(all_y_pred1, axis=0)
        combined_y_label1 = tf.concat(all_y_label1, axis=0)
        if(DATASET == 'IP' or DATASET == 'SA'):
            combined_y_pred2 = tf.concat(all_y_pred2, axis=0)
            combined_y_label2 = tf.concat(all_y_label2, axis=0)
        else:
            combined_y_pred2 = []
            combined_y_label2 = []
    except:
        combined_y_pred1 = []
        combined_y_label1 = []
        combined_y_pred2 = []
        combined_y_label2 = []
    return combined_y_pred1, combined_y_pred2, combined_y_label1, combined_y_label2  # best_mc_predictions1, best_mc_predictions2, best_y1, best_y2

## Main

In [13]:
# Data Related Initializations

def LoadData():
    global X, Y, patches,target_names, NUM_CLASSES, TRAINING_CLASSES, TRAINING_LABELS, TUNNING_LABELS, TESTING_CLASSES, TESTING_LABELS, TRAINING_PATCHES, TUNNING_PATCHES, TESTING_PATCHES, data
    data = Data(DATASET, PCA_COMPONENTS, WINDOW_SIZE)
    X, Y, patches = data.get_data()
    target_names = data.get_target_names()
    NUM_CLASSES, TRAINING_CLASSES, TRAINING_LABELS, TUNNING_LABELS, TESTING_CLASSES, TESTING_LABELS, TRAINING_PATCHES, TUNNING_PATCHES, TESTING_PATCHES = data.load_defaults()

In [14]:
# Model Related Initializations

def LoadModel(encoder:str):
    global model, ProtoModel, optimizer, checkpoint
    encoder = importlib.import_module('encoders.' + encoder)
    model = encoder.createModel(
        IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_DEPTH, IMAGE_CHANNEL)
    ProtoModel = Prototypical(
        model, IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_DEPTH, IMAGE_CHANNEL, MC_LOSS_WEIGHT, TAU, N_TIMES)
    optimizer = Adam(learning_rate=LEARNING_RATE)
    checkpoint = tf.train.Checkpoint(optimizer=optimizer, ProtoModel=model)

### Driver Function

In [15]:
def TAU_test_spl():
    global testingData, test_acc, test_loss
    testingData = []
    test_loss = tf.metrics.Mean(name='test_loss')

    test_acc = tf.metrics.Mean(name='test_accuracy')


def setGlobals(run, tau, n_times, encoder, dataset):
    global finalReportPath, predict_path, model_save_path, report_path, run_folder, reportTotal_path_pre, report_path_pre
    global checkpoint_dir, checkpoint_prefix_train, checkpoint_dir1, checkpoint_prefix_tune, gt_path, predictTrue_path
    global TAU, N_TIMES, ABLATION_FOLDER, trainingData, tunningData, testingData, DATASET, reportTotal_path, current_path
    global train_acc, train_loss, tune_acc, tune_loss, test_acc, test_loss, checkpoint, ProtoModel, optimizer, model
    TAU = tau
    N_TIMES = n_times
    DATASET = dataset

    ABLATION_FOLDER = CWD + '\\ablation\\' + TABLE + '\\'
    run_folder =  f'{date.today()}' + '-' + f'{datetime.now().hour}_5_1' + '\\' 
    checkpoint_dir = CWD + '\\saves\\' + run_folder + DATASET + '\\' + f'{TRAIN_K}_shot_way' + '\\Train'
    checkpoint_prefix_train = os.path.join(checkpoint_dir, "ckpt")
    checkpoint_dir1 =checkpoint_dir + '\\Tune'
    checkpoint_prefix_tune = os.path.join(checkpoint_dir1, "ckpt")
    
    current_path = ABLATION_FOLDER + DATASET + '\\' + f'{encoder}\\{TAU}\\{N_TIMES}' + f'\\{run+1}'
    
    model_save_path = current_path + '_encoder.h5'
    
    report_path = current_path + '_post_tune_report.txt'
    reportTotal_path = current_path + '_post_tune_reportTotal.txt'
    report_path_pre = current_path + '_pre_tune_report.txt'
    reportTotal_path_pre = current_path + '_pre_tune_reportTotal.txt'
    predict_path = current_path + '_prediction.png'
    predictTrue_path = current_path + '_predictionTrue.png'
    gt_path = current_path + '_gt.png'
    finalReportPath = ABLATION_FOLDER + 'report.csv'
    os.makedirs(os.path.dirname(report_path), exist_ok=True)
    trainingData = []
    tunningData = []
    testingData = []
    
    train_loss = tf.metrics.Mean(name='train_loss')

    train_acc = tf.metrics.Mean(name='train_accuracy')

    tune_loss = tf.metrics.Mean(name='tune_loss')

    tune_acc = tf.metrics.Mean(name='tune_accuracy')

    test_loss = tf.metrics.Mean(name='test_loss')

    test_acc = tf.metrics.Mean(name='test_accuracy')
        
    checkpoint = None  # To be used for loading checkpoints. Declared in the Main Block

    ProtoModel = None  # Prototypical Network Object. Declared in the Main Block

    model = None  # Model Object. Declared in the Main Block

    optimizer = None  # Optimizer Object. Declared in the Main Block

@timeIt
def doRun(run, dataset, encoder, tau, n_times):

    if(DATASET != dataset):
        LoadData()

    setGlobals(run, tau, n_times, encoder, dataset)

    LoadModel(encoder)

    print('Training...')
    trainingEpochs(patches, TRAINING_LABELS, TRAINING_EPOCH, TRAINING_EPISODE)
    print()
    if(PRE_TUNE_TESTING):
        print('Pre Tune Testing...')
        mc_predictions1, mc_predictions2, y1, y2 = testingEpochs(TESTING_PATCHES, TESTING_LABELS, TESTING_EPOCH)
        print()

        stats_pre = Stats(mc_predictions1, mc_predictions2, y1, y2)
        
        
        try:
            stats_pre.saveReport(report_path_pre, f'{ABLATION_FOLDER}pre_report.csv',  DATASET, N_TIMES, TAU, run, encoder)
            print('Report Saved!')
        except Exception as e:
            print('Failed to Save Report')
            print(e)
        predictions, predictionsTrue, Y, y_test, y_pred = predictImage(data, ProtoModel, IMAGE_DATA, N_TIMES, 10,TAU, VotingTimes=5, RandomSupport=True)

        try:
            stats1_pre = Stats(None, None, None, None, y_test, y_pred, False)
            stats1_pre.saveReport(reportTotal_path_pre, finalReportPath,  DATASET, N_TIMES, TAU, run, encoder, True)
            print('Image Prediction Report Saved!')
        except Exception as e:
            print('Failed to Save Image Prediction Report')
            print(e)
        
        spectral.save_rgb(f"{current_path}_prediction_pre.png", predictions.astype(int), colors=spectral.spy_colors, format='png')
        spectral.save_rgb(f"{current_path}_predictionTrue_pre.png", predictionsTrue.astype(int), colors=spectral.spy_colors, format='png')
        TAU_test_spl()
    # return 
    print('Tunning...')
    tunningEpochs(TUNNING_PATCHES, TESTING_LABELS,TUNNING_EPOCH, TUNNING_EPISODE)
    print()
    
    print('Testing...')
    mc_predictions1, mc_predictions2, y1, y2 = testingEpochs(TESTING_PATCHES, TESTING_LABELS, TESTING_EPOCH)
    print()

    stats = Stats(mc_predictions1, mc_predictions2, y1, y2)
    
    # stats.printReport()
    
    try:
        stats.saveReport(report_path, finalReportPath,  DATASET, N_TIMES, TAU, run, encoder)
        print('Report Saved!')
    except Exception as e:
        print('Failed to Save Report')
        print(e)

    try:
        # ProtoModel.save(model_save_path)
        print('Model Saved!')
    except Exception as e:
        print("Failed to Save Model")
        print(e)
    
    predictions, predictionsTrue, Y, y_test, y_pred = predictImage(data, ProtoModel, IMAGE_DATA, N_TIMES, 10,TAU, VotingTimes=5, RandomSupport=True)

    try:
        stats1 = Stats(None, None, None, None, y_test, y_pred, False)
        stats1.saveReport(reportTotal_path, finalReportPath,  DATASET, N_TIMES, TAU, run, encoder, True)
        print('Image Prediction Report Saved!')
    except Exception as e:
        print('Failed to Save Image Prediction Report')
        print(e)
    
    spectral.save_rgb(predict_path, predictions.astype(int), colors=spectral.spy_colors, format='png')
    spectral.save_rgb(predictTrue_path, predictionsTrue.astype(int), colors=spectral.spy_colors, format='png')
    # spectral.save_rgb(gt_path, Y.astype(int), colors=spectral.spy_colors, format='png')
    
    

def main():
    # Tweekables:
    global TABLE, PRE_TUNE_TESTING
    TABLE = 'table tr 40 tu 15 adzz' 
    PRE_TUNE_TESTING = True
    done = [] 
    n_timess = [1, 5, 15, 20, 25]
    taus = [1.8]
    datasets = ['IP']
    encoders = ['conv_sa_adzz']
    runs  = 1
    doneRuns = 0 # Increment this if doing more runs for same combinations
    for dataset in datasets:
        global DATASET
        DATASET = dataset
        LoadData() 
        for encoder in encoders:
            for tau in taus: 
                for n_times in n_timess:
                    for run in range(runs):
                        print(f'\n\nRUN: {run+1+doneRuns} | DATASET: {dataset} | ENCODER: {encoder} | TAU: {tau} | N_TIMES: {n_times}\n')
                        try:
                            doRun(run, dataset, encoder, tau, n_times)
                        except Exception as e:
                            print(f"Error: {e}")
                            continue


main()

    



RUN: 1 | DATASET: IP | ENCODER: conv_sa_adzz | TAU: 1.8 | N_TIMES: 25

Training...


Epoch 40/40, Episode 100/100 (Loss: 12.62, Acc: 99.43): 100%|██████████| 4000/4000 [58:56<00:00,  1.13it/s]


Function 'trainingEpochs' executed in 3536.8819s

Pre Tune Testing...


Epochs: 100%|██████████| 1000/1000 [15:36<00:00,  1.07it/s, OA1=0.9333333373069763, OA2=1.0]              


Function 'testingEpochs' executed in 936.9148s

OA 97.10 | KA 96.53 | AA 97.10
Report Saved!


...

Function 'tunningEpochs' executed in 815.3009s

Testing...


Epochs: 100%|██████████| 1000/1000 [15:28<00:00,  1.08it/s, OA1=1.0, OA2=0.9777777791023254]              


Function 'testingEpochs' executed in 928.7250s

OA 99.06 | KA 98.87 | AA 99.06
Report Saved!
Model Saved!


Patch: 225it [09:52,  2.63s/it]                         

OA 92.16 | KA 91.08 | AA 91.53
Image Prediction Report Saved!
Function 'doRun' executed in 7416.0019s


In [16]:
# os.system("shutdown /s /t 2") 